<a href="https://colab.research.google.com/github/sunnykvofm1994/SesnorAndPlannig/blob/main/project2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
from numpy.linalg import inv
import math
import matplotlib.pyplot as plt

In [ ]:
#Read Input data File
measure_giv = pd.read_csv('data_ekf.txt',header=None, delim_whitespace = True, skiprows=1)
# print(measure_giv)

In [ ]:
############ for copying initial readings from first row of input file.###################

prev_time = 1477010443000000/1000000.0                                               #time in microsecond
global x
x = np.array([ [0.312242], [0.5803398], [0], [0], [0], [0] ])                        #x, y, vx, vy, phi, w

############ Initialize variables for storing ground truth and RMSE values ###################3
ground_truth = np.zeros([6, 1])


In [ ]:
############################################### Initialize matrices P and A #################################################
P = np.array([
        [1, 0, 0, 0, 0, 0],
        [0, 1, 0, 0, 0, 0],
        [0, 0, 1000, 0, 0, 0],
        [0, 0, 0, 1000, 0, 0],
        [0, 0, 0, 0, 1000, 0],
        [0, 0, 0, 0, 0, 1000]])                                                       # 1000 just taken randomly as any value >>1
F = np.array([
        [1.0, 0, 1.0, 0, 0, 0],
        [0, 1.0, 0, 1.0, 0, 0],
        [0, 0, 1.0, 0, 0, 0],
        [0, 0, 0, 1.0, 0, 0],
        [0, 0, 0, 0, 1.0, 1.0],
        [0, 0, 0, 0, 0, 1.0]])                                                       # state matrix

var_x = 9
var_y = 9
var_alpha = 9

Q = np.zeros([6, 6])                                                                 # initialized as zero matrix later will be updated

In [ ]:
H_lidar = np.array([
        [1.0, 0, 0, 0, 0, 0],
        [0, 1.0, 0, 0, 0, 0]
        ])
I = np.identity(6)
z_lidar = np.zeros([2, 1])
R_lidar = np.array([
        [0.0225, 0],
        [0, 0.0225]
        ])

In [ ]:
##################################################### jacobian matrix for radar ####################################
def H_j(x):                                                                         
  x_t = x[0][0]
  y_t = x[1][0]
  xdot = x[2][0]
  ydot = x[3][0]

  H_j_radar = np.array([[x_t/pow((x_t**2+y_t**2), 1/2), y_t/pow((x_t**2+y_t**2),1/2), 0, 0, 0, 0],
                             [(y_t*(xdot*y_t-ydot*x_t)/pow((x_t**2+y_t**2),3/2)), (x_t*(ydot*x_t-xdot*y_t)/pow((x_t**2+y_t**2),3/2)), x_t/pow((x_t**2+y_t**2),1/2), y_t/pow((x_t**2+y_t**2),1/2), 0, 0],
                             [0, 0, 0, 0, 1, 0]])
  return  H_j_radar
 
z_radar = np.zeros([3, 1])
R_radar = np.array([
        [0.05, 0, 0],
        [0, 0.05, 0],
        [0, 0, 0.05]
        ])

In [ ]:
 ############################################## prediction using EKF ##################################
def EKF_prediction():                                                               
                                                                                          
  global x, P, Q
  x = np.matmul(F, x)
  Ft = np.transpose(F)
  P = np.add(np.matmul(F, np.matmul(P, Ft)), Q)

In [ ]:
################################################## EKF  sensor measurement update ###############################################

pi = math.pi
w_radar = np.array([
        [0.5], 
        [0.5],
        [2.05]])

w_lidar = np.array([
        [0.05], 
        [0.05]])
def EKF_update(z, sensor):
  global x, P  
    
  if sensor == 'R':
    H = H_j(x)
    z_predict = np.add(np.matmul(H, x),w_radar)
  else:
    H = H_lidar
    z_predict = np.add(np.matmul(H, x),w_lidar)                                                                                 # Measurement update step
  
  Y = np.subtract(z,z_predict)

  if sensor == 'R':
    while(Y[2][0] > pi):
      Y[2][0] = Y[2][0] -2*pi
    while(Y[2][0] < -pi):
      Y[2][0] = Y[2][0] + 2*pi
  

  Ht = np.transpose(H)
  S = np.add(np.matmul(H, np.matmul(P, Ht)), R)
  PHt = np.matmul(P, Ht)
  Si = inv(S)
  K = np.matmul(PHt, Si)
  
                                                                                                         
                                                                                                         # New state
  x = np.add(x, np.matmul(K, Y))
  P = np.matmul(np.subtract(I ,np.matmul(K, H)), P)


In [ ]:
############################################## iterating and updating sensor data ##################################################


for i in range (len(measure_giv)):
    new_measure = measure_giv.iloc[i, :].values
    if new_measure[0] == 'L':
                                                      # Calculate Timestamp and higher order time variables
      cur_time = new_measure[3]/1000000.0
    if new_measure[0] == 'R':
      cur_time = new_measure[4]/1000000.0

    dt = cur_time - prev_time
    prev_time = cur_time
    
    dt_2 = dt * dt
    dt_3 = dt_2 * dt
    dt_4 = dt_3 * dt

                                                       # Update matrix F with dt value
    F[0][2] = dt
    F[1][3] = dt
    F[4][5] = dt

                                                       #Updating Q matrix
    Q[0][0] = dt_4/4*var_x
    Q[0][2] = dt_3/2*var_x
    Q[1][1] = dt_4/4*var_y
    Q[1][3] = dt_3/2*var_y
    Q[2][0] = dt_3/2*var_x
    Q[2][2] = dt_2*var_x
    Q[3][1] = dt_3/2*var_y
    Q[3][3] = dt_2*var_y
    Q[4][4] = dt_4/4*var_alpha
    Q[4][5] = dt_3/2*var_alpha
    Q[5][4] = dt_3/2*var_alpha
    Q[5][5] = dt_2*var_alpha


    if new_measure[0] == 'L':
                                                      #Updating lidar sensor readings
      z_lidar[0][0] = new_measure[1]
      z_lidar[1][0] = new_measure[2]

                                                      #Collecting ground truths
      ground_truth[0] = new_measure[4]
      ground_truth[1] = new_measure[5]
      ground_truth[2] = new_measure[6]
      ground_truth[3] = new_measure[7]
      ground_truth[4] = new_measure[8]
      ground_truth[5] = new_measure[9]

      R = R_lidar

      EKF_prediction()
      EKF_update(z_lidar, new_measure[0])
      
      print("-------------------------------------------------------------------------")
     # print('{:10s} | {:8.3f} | {:8.3f}'.format("lidar values:", z_lidar[0][0],z_lidar[1][0]))
      print('{:10s} | {:8.3f} | {:8.3f} | {:8.3f} | {:8.3f} | {:8.3f} | {:8.3f} |'.format("Lidar Estimated values:", x[0][0], x[1][0], x[2][0], x[3][0], x[4][0], x[5][0] ))
      print('{:10s} | {:8.3f} | {:8.3f} | {:8.3f} | {:8.3f} | {:8.3f} | {:8.3f} |'.format("Ground Truth  :", ground_truth[0][0], ground_truth[1][0], ground_truth[2][0], ground_truth[3][0], ground_truth[4][0], ground_truth[5][0]))
      # print('Lidar Estimated X: ', x)
      # print('Lidar Ground Truth X', ground_truth)

    elif new_measure[0] == 'R':
                                                                          #Updating radar sensor readings
      z_radar[0][0] = new_measure[1]                                      #r
      z_radar[1][0] = new_measure[3]                                      #rdot
      z_radar[2][0] = new_measure[2]                                      #phi

                                                                          #Collecting ground truths
      ground_truth[0] = new_measure[5]
      ground_truth[1] = new_measure[6]
      ground_truth[2] = new_measure[7]
      ground_truth[3] = new_measure[8]
      ground_truth[4] = new_measure[9]
      ground_truth[5] = new_measure[10]

      R = R_radar

      EKF_prediction()
      EKF_update(z_radar, new_measure[0])
      #print(z_radar)
      print("-------------------------------------------------------------------------")
      print('{:10s} | {:8.3f} | {:8.3f} | {:8.3f} | {:8.3f} | {:8.3f} | {:8.3f} |'.format("Radar Estimate:", x[0][0], x[1][0], x[2][0], x[3][0], x[4][0], x[5][0] ))
      print('{:10s} | {:8.3f} | {:8.3f} | {:8.3f} | {:8.3f} | {:8.3f} | {:8.3f} |'.format("Ground Truth  :", ground_truth[0][0], ground_truth[1][0], ground_truth[2][0], ground_truth[3][0], ground_truth[4][0], ground_truth[5][0]))
    #Commented
      '''plt.figure()
      plt.plot(ground_truth)
      plt.legend(["ground truth"])
      plt.plot(z_radar)
      plt.legend(["radar predicted values"])
      plt.show()'''
     # k1=[x[0][0], x[1][0], x[2][0], x[3][0], x[4][0], x[5][0]]
      #k=[z_radar,new_measure,z_lidar,ground_truth]
      #plt.figure()
      #plt.plot(x[0],x[1],label='estmates')
      #plt.plot(ground_truth,)
    